In [2]:
# Utility: Preparation text

from contractions import CONTRACTION_MAP

##========== PREPARATION TEXT ===========##

# Contraction
def expand_contractions(sentence, contraction_mapping=CONTRACTION_MAP):
    """
    Expand the contractions in a sentence. For example don't => do not.
    
    Paramters:
    sentence (str): The input sentence to clean.
    contraction_mapping (dict): A dictionary for mapping contractions.
    
    
    Returns:
    str: The expanded contraction sentence.
    """
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                      flags=re.IGNORECASE|re.DOTALL)
    
    def expanded_match(contraction):
        """
        Filter for expanding the matched contraction.
        
        Parameters:
        contraction (str): The input of contraction
        
        Returns:
        str: The expanded contraction.
        """
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match) if contraction_mapping.get(match) else contraction_mapping.get(match.lower())
        
        expanded_contraction = first_char + expanded_contraction[1:]
        return expanded_contraction
    
    expanded_sentence = contractions_pattern.sub(expanded_match, sentence)
    return expanded_sentence


def remove_extra_spaces(sentence):
    # Use regex to replace multiple spaces with a single space
    return re.sub(r'\s+', ' ', sentence).strip()


def remove_non_ascii(text):
    """
    Remove all non-ASCII characters from the text.

    Parameters:
    text (str): The input text to clean.

    Returns:
    str: The cleaned text with only ASCII characters.
    
    """
    
    return ''.join([char for char in text if ord(char) < 128])

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime

import re
import json

from nltk.tokenize import word_tokenize

import torch
import torch.nn.functional as F
from transformers import BertTokenizer

from tqdm import tqdm

C:\Users\Baha Tegar\Desktop\Preparation\Transformers\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import pipeline

# Load Model

**Summarization**

In [5]:
modelBertSum = pipeline('summarization', model='model/summarization-0', device=0)
modelBertSum

C:\Users\Baha Tegar\Desktop\Preparation\Transformers\.venv\Lib\site-packages\transformers\models\bart\configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


# Load Dataset

In [6]:
with open('temp/representation.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

len(data)

1511

In [7]:
df = pd.DataFrame.from_dict(data)
df = df[['review_id', 'review_time', 'review', 'like']].copy()
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1511 entries, 0 to 1510
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review_id    1511 non-null   int64 
 1   review_time  1511 non-null   object
 2   review       1511 non-null   object
 3   like         1511 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 47.3+ KB
None


,review_id,review_time,review,like
0,0,2024-11-28T13:41:44Z,The cars design is like it came from a cartoon.,0
1,2,2024-11-27T01:56:33Z,Tankz u for being so honest.\nMuch love and re...,0
2,3,2024-11-26T18:27:21Z,Amazing review your really good at this love w...,0
3,5,2024-11-25T18:09:24Z,"Honestly, the Cybertruck might be the first ca...",0
4,6,2024-11-25T03:55:09Z,I will be buying one anytime soon add expensiv...,0


In [8]:
# # df = pd.read_csv('sample-mcd.csv', encoding='latin1')
# df = pd.read_csv('McDonald_s_Reviews.csv', encoding='latin1')
# df = df[['reviewer_id', 'review_time', 'review', 'rating']].copy()
# print(df.info())
# df.head()

In [9]:
# Apply extraction

def process_text(x):
    # Prepare sentence
    texts = expand_contractions(x)
    texts = remove_extra_spaces(x)
    texts = remove_non_ascii(x)
    
    return texts

df['review_processed'] = df['review'].apply(process_text)
df.head()

,review_id,review_time,review,like,review_processed
0,0,2024-11-28T13:41:44Z,The cars design is like it came from a cartoon.,0,The cars design is like it came from a cartoon.
1,2,2024-11-27T01:56:33Z,Tankz u for being so honest.\nMuch love and re...,0,Tankz u for being so honest.\nMuch love and re...
2,3,2024-11-26T18:27:21Z,Amazing review your really good at this love w...,0,Amazing review your really good at this love w...
3,5,2024-11-25T18:09:24Z,"Honestly, the Cybertruck might be the first ca...",0,"Honestly, the Cybertruck might be the first ca..."
4,6,2024-11-25T03:55:09Z,I will be buying one anytime soon add expensiv...,0,I will be buying one anytime soon add expensiv...


# Preprocessing

In [10]:
def is_only_number(text):
    
    pattern = r'^\d+$'
    
    if re.search(pattern, text, re.IGNORECASE):
        return True
    return False

In [11]:
# def extract_date(x):
#     # Get current date
#     current_date = pd.to_datetime("today").date()

#     offset, period, _ = x.split(" ")
#     if is_only_number(offset):
#         offset = int(offset)
#     else:
#         offset = 1

#     if "year" in period:
#         offset = pd.DateOffset(years=offset)
#     elif "month" in period:
#         offset = pd.DateOffset(months=offset)
#     else:
#         offset = pd.DateOffset(days=offset)

#     return (current_date - offset).date().strftime('%Y-%m-%d')

In [12]:
def extract_date(x):
    # Convert to datetime
    datetime_obj = datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ')

    return str(datetime_obj.date())

In [13]:
def extract_rating(x):
    score = x.split(" ")[0]
    return int(score)

In [14]:
df['review_time'] = df['review_time'].apply(extract_date)

In [15]:
# df['rating'] = df['rating'].apply(extract_rating)

In [16]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1511 entries, 0 to 1510
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   review_id         1511 non-null   int64 
 1   review_time       1511 non-null   object
 2   review            1511 non-null   object
 3   like              1511 non-null   int64 
 4   review_processed  1511 non-null   object
dtypes: int64(2), object(3)
memory usage: 59.1+ KB
None


,review_id,review_time,review,like,review_processed
0,0,2024-11-28,The cars design is like it came from a cartoon.,0,The cars design is like it came from a cartoon.
1,2,2024-11-27,Tankz u for being so honest.\nMuch love and re...,0,Tankz u for being so honest.\nMuch love and re...
2,3,2024-11-26,Amazing review your really good at this love w...,0,Amazing review your really good at this love w...
3,5,2024-11-25,"Honestly, the Cybertruck might be the first ca...",0,"Honestly, the Cybertruck might be the first ca..."
4,6,2024-11-25,I will be buying one anytime soon add expensiv...,0,I will be buying one anytime soon add expensiv...


# Paraphrasing

In [17]:
def process(x):
    tokens = word_tokenize(x)
    print("NUM TOKEN: ", len(tokens))
    if len(tokens) > 30:
        set_max = int(0.99 * len(tokens))
        set_min = int(0.25 * len(tokens))
    
        return modelBertSum(f"correct:{x} </s>", min_length=set_min, max_length=set_max)[0]['summary_text']
    return x

In [18]:
# Testing
# for i in range(0, 30):
#     sample = df['review_processed'].iloc[i]
#     print(f"{i}. ", sample)

#     print("SUMMARIZE: ")
#     print(process(sample))
#     print()

In [19]:
tqdm.pandas()
# df['review_processed'] = df['review_processed'].progress_apply(lambda x: modelBertSum(f"correct: {x}</s>")[0]['summary_text'])
df['review_processed'] = df['review_processed'].progress_apply(lambda x: process(x))

  0%|                                                                                         | 0/1511 [00:00<?, ?it/s]

NUM TOKEN:  11
NUM TOKEN:  13
NUM TOKEN:  21
NUM TOKEN:  42


  0%|▎                                                                                | 5/1511 [00:01<06:28,  3.87it/s]

NUM TOKEN:  59


  0%|▎                                                                                | 6/1511 [00:01<07:30,  3.34it/s]

...

NUM TOKEN:  89


100%|█████████████████████████████████████████████████████████████████████████████▉| 1509/1511 [08:29<00:00,  3.01it/s]

NUM TOKEN:  14
NUM TOKEN:  32


100%|██████████████████████████████████████████████████████████████████████████████| 1511/1511 [08:29<00:00,  2.96it/s]

NUM TOKEN:  12


# Save Data

In [20]:
result = df[['review_id', 'review_time', 'like', 'review', 'review_processed']].to_dict('index')
result

{0: {'review_id': 0,
  'review_time': '2024-11-28',
  'like': 0,
  'review': 'The cars design is like it came from a cartoon.',
  'review_processed': 'The cars design is like it came from a cartoon.'},
 1: {'review_id': 2,
  'review_time': '2024-11-27',
  'like': 0,
  'review': 'Tankz u for being so honest.\nMuch love and respect from trinidad',
  'review_processed': 'Tankz u for being so honest.\nMuch love and respect from trinidad'},
 2: {'review_id': 3,
  'review_time': '2024-11-26',
  'like': 0,
  'review': 'Amazing review your really good at this love watching you man I feel like you could review anything. Well done',
  'review_processed': 'Amazing review your really good at this love watching you man I feel like you could review anything. Well done'},
 3: {'review_id': 5,
  'review_time': '2024-11-25',
  'like': 0,
  'review': 'Honestly, the Cybertruck might be the first car I’d ever put a sticker on if I owned one someday. Otherwise, I’d constantly worry about accidentally tryin

In [21]:
with open("temp/temp-1.json", "w") as file:
    json.dump(result, file, indent=4)

In [22]:
# df.to_csv("preprocessed-mcd.csv", index=False)

In [23]:
torch.cuda.reset_peak_memory_stats()  # Reset peak stats for debugging
torch.cuda.empty_cache()  # Clear cache


In [24]:
with open("temp/review-processed.json", 'w', encoding='utf-8') as file:
    json.dump(list(df['review_processed'].values), file, indent=4)